# Part 1: Starting the project and creating the database

## The necessary packages

In [1]:
import pandas as pd
import sqlite3
import requests
import json
from datetime import date, timedelta

## Creation of the SQLite database

In [4]:
con = sqlite3.connect("country_code.db")
cur = con.cursor()
cur.execute("CREATE TABLE Country(Name text, Code text)")
cur.close()

## Data recovery from CSV file

In [5]:
df = pd.read_csv("country_code.csv", delimiter=";", index_col=0, encoding="cp437")
df.head()

,Name,Code
1,Afghanistan,AF
2,Albanie,AL
3,AlgÄrie,DZ
4,Samoa orientales,AS
5,Andorre,AD


In [6]:
print(df.Code[df["Name"]=="Namibie"])

149    NaN
Name: Code, dtype: object


In [7]:
df.Code[df["Name"]=="Namibie"]="NA"

In [8]:
print(df.Code[df["Name"]=="Namibie"])

149    NA
Name: Code, dtype: object


## Insertion of data into the database

In [9]:
cur = con.cursor()
for index, row in df.iterrows():
    cur.execute("INSERT INTO Country(Name, Code) values (?, ?)", row)
cur.close()

# Part 2: Data Recovery from an API

## API test

In [10]:
url = "https://date.nager.at/api"
response = requests.get(url)
# The state of the response
print("API response:",response.status_code)

API response: 200


In [11]:
Json_Files = []
for index, row in df.iterrows():
    response = requests.get("https://date.nager.at/api/v2/publicholidays/2017/"+row["Code"])
    if response.status_code==200:
        data = response.json()
        # Creation of JSON files
        with open(row["Code"]+".json", "w") as json_file:
            json.dump(data, json_file)
            Json_Files.append(row["Code"]+".json")

# Part 3: Data storage in the database

In [12]:
# Creation of a public holiday table
cur = con.cursor()
cur.execute("CREATE TABLE Holidays(countryCode text, name text, localName text, type text, global text, fixed text, date text)")
cur.close()

In [13]:
# Insertion of data
cur = con.cursor()
for file in Json_Files:
    with open(file) as json_file:
        data = json.load(json_file)
        for row in data:
            cur.execute("INSERT INTO Holidays(countryCode, name, localName, type, global, fixed, date) values (?, ?, ?, ?, ?, ?, ?)", [row["countryCode"], row["name"], row["localName"], row["type"], row["global"], row["fixed"], row["date"]])
cur.close()

# Partie 4 : Database data analysis

## The country with the most public holidays in 2017

In [21]:
cur = con.cursor()
cur.execute("SELECT c.name FROM Holidays as h JOIN Country as c WHERE c.Code=h.countryCode GROUP BY countryCode ORDER BY COUNT(countryCode) LIMIT 1")
for row in cur:
    print("The country with the most public holidays in 2017 est:",str(row[0]))
cur.close()

The country with the most public holidays in 2017 est: Andorre


## The country with the fewest public holidays in 2017

In [24]:
cur = con.cursor()
cur.execute("SELECT c.name FROM Holidays as h JOIN Country as c WHERE c.Code=h.countryCode GROUP BY countryCode ORDER BY COUNT(countryCode) DESC LIMIT 1")
for row in cur:
    print("The country with the fewest public holidays in 2017 est:",str(row[0]))
cur.close()

The country with the fewest public holidays in 2017 est: VÄnÄzuela


## Holidays of 2017

In [33]:
# Holidays
Holidays = []
cur = con.cursor()
cur.execute("SELECT DISTINCT(date) FROM Holidays")
for row in cur:
    Holidays.append(row[0])
cur.close()
Holidays

['2017-01-01',
 '2017-01-02',
 '2017-03-14',
 '2017-03-22',
 '2017-04-16',
 '2017-04-17',
 '2017-05-01',
 '2017-10-19',
 '2017-11-28',
 '2017-11-29',
 '2017-12-08',
 '2017-12-25',
 '2017-02-27',
 '2017-02-28',
 '2017-03-24',
 '2017-04-02',
 '2017-04-14',
 '2017-05-25',
 '2017-06-17',
 '2017-06-20',
 '2017-07-09',
 '2017-08-21',
 '2017-10-09',
 '2017-11-20',
 '2017-01-26',
 '2017-03-06',
 '2017-03-13',
 '2017-04-15',
 '2017-04-25',
 '2017-05-29',
 '2017-06-05',
 '2017-06-12',
 '2017-08-07',
 '2017-10-02',
 '2017-12-26',
 '2017-01-06',
 '2017-06-15',
 '2017-08-15',
 '2017-10-26',
 '2017-11-01',
 '2017-01-10',
 '2017-04-01',
 '2017-07-10',
 '2017-08-05',
 '2017-10-12',
 '2017-01-21',
 '2017-04-28',
 '2017-08-01',
 '2017-11-30',
 '2017-01-07',
 '2017-03-08',
 '2017-05-09',
 '2017-07-03',
 '2017-11-07',
 '2017-07-21',
 '2017-11-11',
 '2017-05-22',
 '2017-09-10',
 '2017-09-21',
 '2017-11-19',
 '2017-02-02',
 '2017-06-21',
 '2017-08-02',
 '2017-08-06',
 '2017-11-02',
 '2017-07-01',
 '2017-07-

## The days of the year 2017

In [34]:
# Days of the year 2017
daysOfYear = []
days = date(2017, 1, 1)
while days.year == 2017:
    daysOfYear.append(str(days))
    days += timedelta(days=1)
daysOfYear

['2017-01-01',
 '2017-01-02',
 '2017-01-03',
 '2017-01-04',
 '2017-01-05',
 '2017-01-06',
 '2017-01-07',
 '2017-01-08',
 '2017-01-09',
 '2017-01-10',
 '2017-01-11',
 '2017-01-12',
 '2017-01-13',
 '2017-01-14',
 '2017-01-15',
 '2017-01-16',
 '2017-01-17',
 '2017-01-18',
 '2017-01-19',
 '2017-01-20',
 '2017-01-21',
 '2017-01-22',
 '2017-01-23',
 '2017-01-24',
 '2017-01-25',
 '2017-01-26',
 '2017-01-27',
 '2017-01-28',
 '2017-01-29',
 '2017-01-30',
 '2017-01-31',
 '2017-02-01',
 '2017-02-02',
 '2017-02-03',
 '2017-02-04',
 '2017-02-05',
 '2017-02-06',
 '2017-02-07',
 '2017-02-08',
 '2017-02-09',
 '2017-02-10',
 '2017-02-11',
 '2017-02-12',
 '2017-02-13',
 '2017-02-14',
 '2017-02-15',
 '2017-02-16',
 '2017-02-17',
 '2017-02-18',
 '2017-02-19',
 '2017-02-20',
 '2017-02-21',
 '2017-02-22',
 '2017-02-23',
 '2017-02-24',
 '2017-02-25',
 '2017-02-26',
 '2017-02-27',
 '2017-02-28',
 '2017-03-01',
 '2017-03-02',
 '2017-03-03',
 '2017-03-04',
 '2017-03-05',
 '2017-03-06',
 '2017-03-07',
 '2017-03-

## The days that are not public holidays

In [35]:
for day in Holidays:
    daysOfYear.remove(day)
Non_holidays = Holidays
print("The days that are not public holidays are: ",*Non_holidays, sep = "\n")

The days that are not public holidays are: 
2017-01-01
2017-01-02
2017-03-14
2017-03-22
2017-04-16
2017-04-17
2017-05-01
2017-10-19
2017-11-28
2017-11-29
2017-12-08
2017-12-25
2017-02-27
2017-02-28
2017-03-24
2017-04-02
2017-04-14
2017-05-25
2017-06-17
2017-06-20
2017-07-09
2017-08-21
2017-10-09
2017-11-20
2017-01-26
2017-03-06
2017-03-13
2017-04-15
2017-04-25
2017-05-29
2017-06-05
2017-06-12
2017-08-07
2017-10-02
2017-12-26
2017-01-06
2017-06-15
2017-08-15
2017-10-26
2017-11-01
2017-01-10
2017-04-01
2017-07-10
2017-08-05
2017-10-12
2017-01-21
2017-04-28
2017-08-01
2017-11-30
2017-01-07
2017-03-08
2017-05-09
2017-07-03
2017-11-07
2017-07-21
2017-11-11
2017-05-22
2017-09-10
2017-09-21
2017-11-19
2017-02-02
2017-06-21
2017-08-02
2017-08-06
2017-11-02
2017-07-01
2017-07-17
2017-09-30
2017-10-01
2017-04-21
2017-09-07
2017-11-15
2017-03-03
2017-05-06
2017-05-24
2017-09-06
2017-09-22
2017-12-24
2017-02-13
2017-02-20
2017-03-17
2017-04-23
2017-06-24
2017-07-12
2017-09-04
2017-05-21
2017-06-07